# ⚛️ H₂ Ground-State VQE (Noisy Simulation)
## Classical Optimizer Comparison + Noisy Density-Matrix Analysis

This notebook explores the **Variational Quantum Eigensolver (VQE)** applied to
the hydrogen molecule **H₂**, simulated on a **noisy quantum device**.  

The goal is to understand:

- How different **classical optimizers** behave under noise  
- How noise affects **convergence** and **state populations**  
- How to reconstruct and analyse the **final noisy density matrix**  

The notebook uses your production-ready packages:

- `vqe.core.run_vqe_optimizer_comparison`  
- `vqe.core.run_vqe`  
- `vqe_qpe_common.plotting`  

which automatically handle:

- Caching  
- Noise injection  
- Device creation  
- Reproducibility  
- Figure saving  

---

## 🧪 Noise Model (NISQ-inspired)

For each qubit:

- **Depolarizing noise:** 10%  
- **Amplitude damping:** 10%  

These values intentionally exaggerate realistic NISQ noise so that behaviour
differences between optimizers become clearly visible.


In [ ]:
# Core scientific stack
import pennylane as qml
from pennylane import numpy as np
from pennylane import qchem
import matplotlib.pyplot as plt

import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

# VQE engine + plotting utilities
from vqe.core import run_vqe_optimizer_comparison, run_vqe
from vqe_qpe_common.plotting import IMG_DIR

# Molecular system
symbols = ["H", "H"]
coordinates = np.array([
    [0.0, 0.0, 0.0],
    [0.0, 0.0, 0.7414]
])
basis = "sto-3g"

# Noise parameters
depolarizing_prob = 0.1
amplitude_damping_prob = 0.1

seed = 0

# 🔧 Part 1 — Noisy Optimizer Comparison

We compare three optimizers under identical noise conditions:

- `GradientDescent`  
- `Adam`  
- `Momentum`  

Using:

```python
run_vqe_optimizer_comparison(...)
```

This function:

✔ Creates the noisy device  
✔ Applies depolarizing + amplitude-damping channels  
✔ Runs each optimizer  
✔ Handles caching  
✔ Generates and saves a convergence plot  
✔ Displays the figure  

The returned object reports the **final energies** for each optimizer.

In [ ]:
optimizers = ["GradientDescent", "Adam", "Momentum"]

comparison = run_vqe_optimizer_comparison(
    molecule="H2",
    ansatz_name="TwoQubit-RY-CNOT",
    optimizers=optimizers,
    steps=75,
    stepsize=0.1,
    noisy=True,
    depolarizing_prob=depolarizing_prob,
    amplitude_damping_prob=amplitude_damping_prob,
    force=False,
    show=True
)

comparison

# ⭐ Part 2 — Identify the Worst Optimizer

Because noise degrades the optimization landscape, different optimizers
converge to different energies.

We now select the **worst-performing optimizer**, defined as the one with the
**highest final energy**.

In [ ]:
worst_optimizer = max(
    comparison["final_energies"],
    key=comparison["final_energies"].get
)

print(f"🏆 Worst optimizer under noise: {worst_optimizer}")

# 🔍 Part 3 — Full Noisy VQE Using the Worst Optimizer

We now run a complete noisy VQE workflow with:

- **Ansatz:** `TwoQubit-RY-CNOT`  
- **Optimizer:** worst performer  
- **Noise:** 10% depolarizing + 10% amplitude damping  
- **Steps:** 75  

The call below returns:

- `final_state_real` / `final_state_imag` (density matrix)  
- `energies`  
- full configuration & metadata  

We reconstruct the density matrix and analyse its diagonal populations.

In [ ]:
res = run_vqe(
    molecule="H2",
    ansatz_name="TwoQubit-RY-CNOT",
    optimizer_name=worst_optimizer,
    n_steps=75,
    stepsize=0.1,
    noisy=True,
    depolarizing_prob=depolarizing_prob,
    amplitude_damping_prob=amplitude_damping_prob,
    plot=False,
    seed=0,
    force=False
)

rho = np.array(res["final_state_real"]) + 1j * np.array(res["final_state_imag"])
diag_elements = np.diag(rho)
diag_elements

# 🧮 Part 4 — Ket-Form Approximation from the Noisy Density Matrix

For mixed states, the diagonal entries of ρ correspond to **basis-state
populations** after noise.

We extract terms above a threshold and build a compact approximate ket
representation:


In [ ]:
threshold = 1e-2
num_wires = int(np.log2(len(diag_elements)))

ket_terms = []
for idx, amp in enumerate(diag_elements):
    if abs(amp) > threshold:
        ket_terms.append(f"({amp:.4f}|{idx:0{num_wires}b}⟩)")

ket_notation = " + ".join(ket_terms)

print("Approximate noisy state:")
print(f"|ψ⟩ ≈ {ket_notation}")

# 📊 Part 5 — Basis-State Population Plot

A bar plot of the **diagonal entries of ρ** shows which computational basis
states survive under noise.

- Blue bars = real population  
- Orange bars = imaginary component  

Imaginary parts are typically very small but are included for completeness.

In [ ]:
indices = np.where(abs(diag_elements) > threshold)[0]
values = diag_elements[indices]
labels = [f"|{i:0{num_wires}b}⟩" for i in indices]

plt.figure(figsize=(10, 5))
plt.bar(labels, values.real, label="Real")
plt.bar(labels, values.imag, bottom=values.real, alpha=0.6, label="Imag")

plt.xlabel("Basis state")
plt.ylabel("Population")
plt.title("H₂ — Noisy VQE Ground State (Diagonal of ρ)")
plt.legend()
plt.tight_layout()

plt.savefig(os.path.join(IMG_DIR, "H2_Noisy_Ground_State.png"), dpi=300)
plt.show()